In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
df = pd.read_csv('Samara_flat_data12.csv')
df.head()

## Переименовываем колонки

In [ ]:
df = df.rename(
            columns={
                "URL": "url",
                "Цена": "price",
                "Цена за кв.метр": "price_per_sqm",
                "Количество собственников": "number_of_owners",
                "Район": "district",
                "Улица": "street",
                "Вид сделки": "deal_type",
                "Количество комнат": "number_of_rooms",
                "Общая площадь": "total_area",
                "Площадь кухни": "kitchen_area",
                "Жилая площадь": "living_area",
                "Этаж": "floor",
                "Этажей в доме": "floors_in_building",
                "Балкон или лоджия": "balcony_or_loggia",
                "Высота потолков": "ceiling_height",
                "Санузел": "bathroom",
                "Окна": "windows",
                "Ремонт": "renovation",
                "Техника": "appliances",
                "Способ продажи": "sale_method",
                "Тип дома": "building_type",
                "Год постройки": "year_built",
                "Пассажирский лифт": "passenger_elevator",
                "Грузовой лифт": "freight_elevator",
                "В доме": "in_building",
                "Двор": "courtyard",
                "Парковка": "parking",
                "Дополнительно": "additional_info",
                "Тип комнат": "room_type",
                "Мебель": "furniture",
                "Тип продавца": "seller_type",
                "Data_added": "data_added",
            }
        )

## EDA

### Первичный осмотр

In [ ]:
df.shape

In [ ]:
df.dtypes.value_counts()

In [ ]:
# Исследуем количество пропусков
null_count = df.isnull().sum()
null_percentage = (df.isnull().sum() / len(df)) * 100
null_values = pd.DataFrame({
    'column_name': df.columns,
    'null_count': null_count.values,
    'null_percentage': null_percentage.values
}).set_index('column_name').T
null_values

In [ ]:
#Сводная статистика по числовым характеристикам
numerical_features = df.select_dtypes(include=[np.number])
numerical_features.describe().round(3)

In [ ]:
#Сводная статистика по категориальным признакам
categorical_features = df.select_dtypes(include=[object])
categorical_features.describe()

### Поиск аномалий

#### Общая площадь

In [ ]:
plt.figure(figsize=(16, 10))
plt.hist(df['total_area'], bins=50,)
plt.title('Распределение общей площади')
plt.xlabel('Общая площадь')
plt.ylabel('Количество')
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.boxplot(df['total_area'])
plt.title('BoxPlot для общей площади')
plt.ylabel('Общая площадь')
plt.show()

In [ ]:
total_area_outliers = df[(df['total_area'] > 600)]
total_area_outliers

In [ ]:
df = df.drop([310, 510, 4435], axis=0)

#### Жилая площадь

In [ ]:
plt.figure(figsize=(16, 10))
plt.hist(df['living_area'], bins=50,)
plt.title('Распределение жилой площади')
plt.xlabel('Жилая площадь')
plt.ylabel('Количество')
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.boxplot(df['living_area'].dropna())
plt.title('BoxPlot для жилой площади')
plt.ylabel('Жилая площадь')
plt.show()

In [ ]:
living_area_outliers = df[(df['living_area'] > 150)]
living_area_outliers

#### Площадь кухни

In [ ]:
plt.figure(figsize=(16, 10))
plt.hist(df['kitchen_area'], bins=50,)
plt.title('Распределение площади кухни')
plt.xlabel('Площадь кухни')
plt.ylabel('Количество')
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.boxplot(df['kitchen_area'].dropna())
plt.title('BoxPlot для площади кухни')
plt.ylabel('Площадь кухни')
plt.show()

#### Высота потолков

In [ ]:
plt.figure(figsize=(16, 10))
plt.hist(df['ceiling_height'], bins=50,)
plt.title('Распределение высоты потолков')
plt.xlabel('Высота потолков')
plt.ylabel('Количество')
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.boxplot(df['ceiling_height'].dropna())
plt.title('BoxPlot для высоты потолков')
plt.ylabel('Высота потолков')
plt.show()

In [ ]:
def normalize_ceiling_height(df):
    """
    Преобразует значения в столбце 'ceiling_height' так, чтобы первая цифра числа
    стала целой частью, а все остальные - дробной частью.
    Например: 2700.0 -> 2.7, 280.0 -> 2.8, 28.0 -> 2.8
    """
    def _convert_value(x):
        if x == 0:
            return 0.0
        try:
            # Преобразуем число в строку и удаляем точку (если это float)
            s = str(x).replace('.', '')
            # Первая цифра - целая часть, остальные - дробная
            if len(s) > 1:
                return float(f"{s[0]}.{s[1:]}")
            else:
                return float(s[0])
        except:
            return x
    
    df['ceiling_height'] = df['ceiling_height'].apply(_convert_value)
    return df

In [ ]:
df = normalize_ceiling_height(df)

In [ ]:
df.describe().round(3)

## Исследуем зависимую переменную

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['price'], bins=50,)
plt.show()

correlation = df.corr()['price'].sort_values(ascending=False)
print(correlation)

## Обработка колонок
### Модификация фичей и заполнение пропусков

In [ ]:
null_count = df.isnull().sum()
null_percentage = (df.isnull().sum() / len(df)) * 100
null_values = pd.DataFrame({
    'column_name': df.columns,
    'null_count': null_count.values,
    'null_percentage': null_percentage.values
}).set_index('column_name').T
null_values

#### one_hot_encoding

In [ ]:
# Функция для one_hot_encoding колонок со списками
def one_hot_encoding(column_name):
    values_list = df[column_name].dropna()
    el_list = []
    for val in values_list:
        val_el = val.split(', ')
        for el in val_el:
            if el not in el_list:
                el_list.append(el)
    for el_l in el_list:
        df[el_l] = df[column_name].str.contains(el_l, case=False, na=False).astype(int)

#### Количество собственников

In [ ]:
df['number_of_owners'].nunique()

In [ ]:
df['number_of_owners'].unique().tolist()

In [ ]:
# Колонку "количество собственников" преобразуем в "собственник или застройщик"
df.loc[df['number_of_owners'].isin(['1 собственник', '2 собственника или больше', 'Контактное лицо']), 'owner_or_developer'] = 1
df['owner_or_developer'] = df['owner_or_developer'].fillna(0)
df['owner_or_developer'] = df['owner_or_developer'].astype(int)

#### Район

In [ ]:
df['district'].nunique()

In [ ]:
df['district'] = df['district'].fillna('unknown')

#### Вид сделки

In [ ]:
df['deal_type'].value_counts()

In [ ]:
df['deal_type'] = df['deal_type'].fillna('unknown')

#### Количество комнат

In [ ]:
# Колонку "количество комнат" преобразуем в порядковый признак. Студии и квавртиры со свободной планировкой относим к однушкам
df.loc[df['number_of_rooms']=='студия', 'number_of_rooms'] = 1
df.loc[df['number_of_rooms']=='свободная планировка', 'number_of_rooms'] = 1
df['number_of_rooms'] = df['number_of_rooms'].astype(int)

#### Общая площадь

In [ ]:
# Общая площадь сильно коррелирует с таргетом, поэтому удаляем строчки с пропущенными значениями
df = df.dropna(subset=['total_area'], axis=0)

#### Площадь кухни

In [ ]:
# Заполняем пропущенные значения площади кухни на основе общей площади с помощью линеной регрессии

known_data = df[df['kitchen_area'].notna()]
unknown_data = df[df['kitchen_area'].isna()]

model = LinearRegression()
model.fit(known_data[['total_area']], known_data['kitchen_area'])
    
# Предсказываем пропущенные значения
predicted_values = model.predict(unknown_data[['total_area']])
    
# Заполняем пропущенные значения
df.loc[df['kitchen_area'].isna(), 'kitchen_area'] = predicted_values

#### Жилая площадь

In [ ]:
# Заполняем пропущенные значения жилой площади на основе общей площади с помощью линеной регрессии

known_data = df[df['living_area'].notna()]
unknown_data = df[df['living_area'].isna()]

model = LinearRegression()
model.fit(known_data[['total_area']], known_data['living_area'])
    
# Предсказываем пропущенные значения
predicted_values = model.predict(unknown_data[['total_area']])
    
# Заполняем пропущенные значения
df.loc[df['living_area'].isna(), 'living_area'] = predicted_values

#### Балкон или лоджия

In [ ]:
df['balcony_or_loggia'].value_counts()

In [ ]:
# Применяем "one hot encoding" к колонке "балкон или лоджия"
df['loggia'] = df['balcony_or_loggia'].str.contains('лоджия', case=False, na=False).astype(int)
df['balcony'] = df['balcony_or_loggia'].str.contains('балкон', case=False, na=False).astype(int)

#### Высота потолков

In [ ]:
# Заполняем пропущенные значения высоты потолков на основе типа здания
df["ceiling_height"] = (df["ceiling_height"].fillna(df.groupby("building_type")["ceiling_height"].transform("mean")))

#### Ванная комната

In [ ]:
df['bathroom'].value_counts()

In [ ]:
# Заполняем пропущенные значения ванной комнаты на основе количества комнат
df.loc[(df['bathroom'].isna()) & (df['number_of_rooms'].isin(['студия', '1', '2', 'свободная планировка'])), 'bathroom'] = 'совмещенный'
df['bathroom'] = df['bathroom'].fillna('раздельный')

#### Окна

In [ ]:
df['windows'].value_counts()

In [ ]:
# Применяем one_hot_encoding к колонке с окнами
one_hot_encoding('windows')

#### Ремонт

In [ ]:
df['renovation'].value_counts()

In [ ]:
# Заполняем пропуски в колонке ремонт значением "unknown"
df['renovation'] = df['renovation'].fillna('unknown')

#### Техника

In [ ]:
df['appliances'].value_counts()

In [ ]:
# Преобразуем колонку с техникой в бинарную
df['appliances'] = df['appliances'].notna().astype(int)
df['appliances'].value_counts()

#### Способ продажи

In [ ]:
df['sale_method'].value_counts()

In [ ]:
# Заполняем пропуски в колонке "способ продажи" значением "unknown"
df['sale_method'] = df['sale_method'].fillna('unknown')

#### Тип здания

In [ ]:
df['building_type'].value_counts()

In [ ]:
# Заполняем пропуски в колонке "тип здания" значением "unknown"
df['building_type'] = df['building_type'].fillna('unknown')

#### Год постройки

In [ ]:
# # Заполняем пропуски в колонке "год постройки" средним
df['year_built'] = df['year_built'].fillna(df['year_built'].mean()).round().astype(int)

#### Пасажирский лифт

In [ ]:
df['passenger_elevator'].value_counts()

In [ ]:
# Колонку "пасажирский лифт" преобразуем в порядковый признак
df.loc[df['passenger_elevator']=='нет', 'passenger_elevator'] = 0
df['passenger_elevator'] = df['passenger_elevator'].fillna(0)
df['passenger_elevator'] = df['passenger_elevator'].astype(int)

#### Грузовой лифт

In [ ]:
df['freight_elevator'].value_counts()

In [ ]:
# Колонку "грузовой лифт" преобразуем в порядковый признак
df.loc[df['freight_elevator']=='нет', 'freight_elevator'] = 0
df['freight_elevator'] = df['freight_elevator'].fillna(0)
df['freight_elevator'] = df['freight_elevator'].astype(int)

#### В доме

In [ ]:
df['in_building'].value_counts()

In [ ]:
# Применяем "one hot encoding" к колонке "в доме"
df['gas'] = df['in_building'].str.contains('газ', case=False, na=False).astype(int)
df['concierge'] = df['in_building'].str.contains('консьерж', case=False, na=False).astype(int)
df['garbage_chute'] = df['in_building'].str.contains('мусоропровод', case=False, na=False).astype(int)

#### Двор

In [ ]:
df['courtyard'].value_counts()

In [ ]:
# Применяем "one hot encoding" к колонке "двор"
one_hot_encoding('courtyard')

#### Двор

In [ ]:
df['parking'].value_counts()

#### Парковка

In [ ]:
# Применяем "one hot encoding" к колонке "парковка"
one_hot_encoding('parking')

#### Дополнительно

In [ ]:
df['additional_info'].value_counts()

In [ ]:
# Применяем "one hot encoding" к колонке "дополнительно"
one_hot_encoding('additional_info')

#### Тип комнат

In [ ]:
df['room_type'].value_counts()

In [ ]:
# Применяем "one hot encoding" к колонке "тип комнат"
one_hot_encoding('room_type')

#### Мебель

In [ ]:
df['furniture'].value_counts()

In [ ]:
# Применяем "one hot encoding" к колонке "мебель"
one_hot_encoding('furniture')

#### Тип продавца

In [ ]:
df['seller_type'].value_counts()

In [ ]:
# Преобразуем колонку с "тип продвца" в бинарную: 0-агентство, 1-частное лицо
df.loc[df['seller_type']=='агентство', 'seller_type'] = 0
df.loc[df['seller_type']=='частное лицо', 'seller_type'] = 1
df['seller_type'] = df['seller_type'].astype(int)

#### Создаем новые колонки

In [ ]:
df['average_room_size'] = df['living_area'] / df['number_of_rooms']

In [ ]:
df['room_density'] = df['total_area'] / df['number_of_rooms']

In [ ]:
df['living_area_ratio'] = df['living_area'] / df['total_area']

In [ ]:
df['kitchen_area_ratio'] = df['kitchen_area'] / df['total_area']

In [ ]:
df['floor_ratio'] = df['floor'] / df['floors_in_building']

#### Создаем новый признак расстояние до центра города

In [ ]:
# Загружаем координаты центра города
point1 = pd.read_json('center_coords.json')
point1 = (point1.loc[0,'latitude'], point1.loc[0,'longitude'])

In [ ]:
# Cоздаем функцию по расчету расстояния до центра
def calculate_distance(row):
    if not(pd.isnull(row['latitude']) and pd.isnull(row['longitude'])): 
        point2 = (row['latitude'], row['longitude'])
        dist = geodesic(point1, point2).kilometers
        return round(dist,2)
    else:
        return None

In [ ]:
# Расчитываем расстояние до центра города
df['center_distance'] = df.apply(calculate_distance, axis=1)

In [ ]:
df = df.dropna(subset=['center_distance'], axis=0)

### Удаляем лишние колонки

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'id', 'price_per_sqm', 'data_added', 'street'])

In [ ]:
df = df.drop(columns=['number_of_owners', 'deal_type', 'balcony_or_loggia', 'in_building', 'courtyard', 'year_built',
                     'parking', 'windows', 'additional_info', 'room_type', 'furniture', 'latitude', 'longitude'])

In [ ]:
df = df.dropna(subset=['ceiling_height'], axis=0)

In [ ]:
# Удаляем строчки с пропущенными значениями таргета
df = df.dropna(subset=['price'], axis=0)

In [ ]:
# Преобразуем колонку с "цена" в вещественную
df['price'] = df['price'].astype(int)

In [ ]:
null_count = df.isnull().sum()
null_percentage = (df.isnull().sum() / len(df)) * 100
null_values = pd.DataFrame({
    'column_name': df.columns,
    'null_count': null_count.values,
    'null_percentage': null_percentage.values
}).set_index('column_name').T
null_values

In [ ]:
df.dtypes.value_counts()

In [ ]:
df['price'].dtype

### Удаляем выбросы в target

In [ ]:
r = df['price'].quantile(0.75) - df['price'].quantile(0.25) 
upper_quantile = df['price'].quantile(0.75)+1.5 * r
df[df['price']>upper_quantile].shape
df = df[(df['price']<upper_quantile)]

### Обрабатываем категориальные признаки

In [ ]:
categorical_columns = df.select_dtypes(include=[object])
categorical_columns.describe()

In [ ]:
%pip install category_encoders
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

for col in categorical_columns:
    
    # Для колонок с маленькой размерностью — OneHotEncoder
    if df[col].nunique() <= 10:
        # Создаем и применяем OneHotEncoder (drop_first=True для удаления первого уровня)
        encoder = OneHotEncoder(drop='first', sparse_output=False)
        encoded_data = encoder.fit_transform(df[[col]])
        
        # Получаем имена новых колонок
        new_columns = encoder.get_feature_names_out([col])
        
        # Создаем DataFrame с новыми колонками
        encoded_df = pd.DataFrame(encoded_data, columns=new_columns, index=df.index)
        
        # Удаляем исходную колонку и добавляем закодированные
        df = df.drop(col, axis=1)
        df = pd.concat([df, encoded_df], axis=1)
        
    # Для остальных — TargetEncoder
    else:
        encoder = TargetEncoder()
        df[col] = encoder.fit_transform(df[[col]], df['price'])

In [ ]:
df.dtypes.value_counts()

### Строим матрицу корреляций

In [ ]:
mask = np.zeros_like(df.corr())
triangle_indices = np.triu_indices_from(mask)
mask[triangle_indices] = True
mask

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df.corr(), mask=mask, annot=True, annot_kws={"size": 7})
sns.set_style('white')
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
plt.show()

### Смотрим на зависимост таргета от общей площади

In [ ]:
df.plot.scatter(x='total_area', y='price')

### Разделяем выборку на трейн и тест

In [ ]:
X = df.drop('price', axis=1)
y =  df['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Пробуем обучить модель на тестовых параметрах

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from math import sqrt

model = XGBRegressor(
    objective = "reg:squarederror",
    n_estimators = 300,
    max_depth=6,
    learning_rate=0.1,
    random_seed=42,
    subsample = 0.8,
    colsample_bytree = 0.8
)

model.fit(X_train, y_train)

### Смотрим на метрики нашей тестовой модели

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np

# Делаем предсказания на тренировочных и тестовых данных
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Вычисляем метрики для тренировочного набора
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mape_train = mean_absolute_percentage_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Для Adjusted R2
n_train = X_train.shape[0]  # количество наблюдений
p_train = X_train.shape[1]  # количество признаков
adjusted_r2_train = 1 - (1 - r2_train) * (n_train - 1) / (n_train - p_train - 1)

# Вычисляем метрики для тестового набора
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Для Adjusted R2
n_test = X_test.shape[0]  # количество наблюдений
p_test = X_test.shape[1]  # количество признаков
adjusted_r2_test = 1 - (1 - r2_test) * (n_test - 1) / (n_test - p_test - 1)

# Выводим результаты
print("Метрики на тренировочном наборе:")
print(f"RMSE: {rmse_train:.4f}")
print(f"MAPE: {mape_train:.4f}")
print(f"R2: {r2_train:.4f}")
print(f"Adjusted R2: {adjusted_r2_train:.4f}")
print()
print("Метрики на тестовом наборе:")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAPE: {mape_test:.4f}")
print(f"R2: {r2_test:.4f}")
print(f"Adjusted R2: {adjusted_r2_test:.4f}")

### Оптимизируем гиперпараметры модели с помощью optuna

In [ ]:
import optuna
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

def objective(trial):
    # Определяем гиперпараметры для оптимизации
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
        'random_seed': 42,
        'importance_type': "gain"
    }
    
    # Создаем и обучаем модель
    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    
    # Делаем предсказания
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Вычисляем RMSE
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    # Соотношение ошибок (чем ближе к 1, тем лучше)
    ratio = max(rmse_train / rmse_test, rmse_test / rmse_train)
    
    # Комбинированная метрика
    combined_score = rmse_test * ratio
    
    return combined_score
    

# Оптимизация
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

# Лучшая модель
best_params = study.best_params
best_model = XGBRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Финальная оценка
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train RMSE: {rmse_train:.4f}")
print(f"Test RMSE: {rmse_test:.4f}")
print(f"Difference: {abs(rmse_train - rmse_test):.4f}")

### Смотрим на метрики модели с лучшими гиперпараметрами

In [ ]:
# Делаем предсказания на тренировочных и тестовых данных
y_train_pred_best = best_model.predict(X_train)
y_test_pred_best = best_model.predict(X_test)

# Вычисляем метрики для тренировочного набора
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred_best))
mape_train = mean_absolute_percentage_error(y_train, y_train_pred_best)
r2_train = r2_score(y_train, y_train_pred_best)

# Для Adjusted R2
n_train = X_train.shape[0]  # количество наблюдений
p_train = X_train.shape[1]  # количество признаков
adjusted_r2_train = 1 - (1 - r2_train) * (n_train - 1) / (n_train - p_train - 1)

# Вычисляем метрики для тестового набора
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred_best))
mape_test = mean_absolute_percentage_error(y_test, y_test_pred_best)
r2_test = r2_score(y_test, y_test_pred_best)

# Для Adjusted R2
n_test = X_test.shape[0]  # количество наблюдений
p_test = X_test.shape[1]  # количество признаков
adjusted_r2_test = 1 - (1 - r2_test) * (n_test - 1) / (n_test - p_test - 1)

# Выводим результаты
print("Метрики на тренировочном наборе:")
print(f"RMSE: {rmse_train:.4f}")
print(f"MAPE: {mape_train:.4f}")
print(f"R2: {r2_train:.4f}")
print(f"Adjusted R2: {adjusted_r2_train:.4f}")
print()
print("Метрики на тестовом наборе:")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAPE: {mape_test:.4f}")
print(f"R2: {r2_test:.4f}")
print(f"Adjusted R2: {adjusted_r2_test:.4f}")

### Посмотрим на вклад фичей в итоговую модель

In [ ]:
def plot_feature_importance(importance, names, model_type):
    
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    #Define size of bar plot
    plt.figure(figsize=(10,8))
    
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    
    #Add chart labels
    plt.title(model_type + ' FEATURE IMPORTANCE')
    plt.xlabel('Feature importance')
    plt.ylabel('Feature names')

In [ ]:
# Посмотрим на вклад каждой из фичей
plot_feature_importance(best_model.feature_importances_, X_train.columns, 'XGBoost')